In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt



In [2]:
pip install prophet

In [3]:
df=pd.read_csv('covid.csv')

FileNotFoundError: ignored

In [ ]:
df.head()

In [ ]:
df.rename(columns={'Date':'date','Lat':'lat','Province/State':'state'},inplace=True)

In [ ]:
df.head()

In [ ]:
#records from last date
top=df[df['date']==df['date'].max()].reset_index()

In [ ]:
world=top.groupby(by='Country/Region')['Confirmed','Deaths','Recovered','Active'].sum().reset_index()
world

In [ ]:
world.sort_values(['Active','Deaths'],ascending=False)[:5]

In [ ]:
import plotly
import plotly.express as px

In [ ]:
figure=px.choropleth(world,locations='Country/Region',locationmode='country names',color='Recovered',hover_name='Country/Region',range_color=[1,12000])
figure.show()

In [ ]:
total_cases=df.groupby(by='date')['Confirmed','Active'].sum().reset_index()

In [ ]:
total_cases

In [ ]:
total_cases['date']=pd.to_datetime(total_cases['date'])

In [ ]:
total_cases.info()


In [ ]:
import seaborn as sns
plt.figure(figsize=(40,10))
ax=sns.pointplot(x=total_cases.date.dt.date,y=total_cases['Confirmed'])
ax.set(xlabel='date',ylabel='confirmed')
plt.xticks(rotation=90)
plt.show()



In [ ]:
top_actives=top.groupby(by='Country/Region')['Active'].sum().sort_values(ascending=False)[:20].reset_index()

In [ ]:
top_actives

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(x=top_actives.Active,y=top_actives['Country/Region'])
ax.set(xlabel='Active_cases',ylabel='Country')

In [ ]:
top_recovered=top.groupby('Country/Region')['Recovered'].sum().sort_values(ascending=False)[:20].reset_index()

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(x=top_recovered.Recovered,y=top_recovered['Country/Region'])
ax.set(xlabel='Total_Recovered',ylabel='Country')

In [ ]:
top_recovered=top.groupby('Country/Region')['Recovered'].sum().sort_values(ascending=False)[:20].reset_index()

In [ ]:
highest_deaths=top.groupby('Country/Region')['Deaths'].sum().sort_values(ascending=False)[:20].reset_index()

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(x=highest_deaths.Deaths,y=highest_deaths['Country/Region'])
ax.set(xlabel='Total_deaths',ylabel='Country')

In [ ]:
top_confirmed=top.groupby('Country/Region')['Confirmed'].sum().sort_values(ascending=False)[:20].reset_index()

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(x=top_confirmed.Confirmed,y=top_confirmed['Country/Region'])
ax.set(xlabel='Total_ConfirmedCases',ylabel='Country')

In [ ]:
india=df[df['Country/Region']=='India']

In [ ]:
india=india.groupby('date')['Active','Recovered','Deaths','Confirmed'].sum().reset_index()


In [ ]:
china=df[df['Country/Region']=='China']
china=china.groupby('date')['Active','Recovered','Deaths','Confirmed'].sum().reset_index()

In [ ]:
us=df[df['Country/Region']=='US']
us=us.groupby('date')['Active','Recovered','Deaths','Confirmed'].sum().reset_index()

In [ ]:
india

In [ ]:
#Confirned cases trend
plt.figure(figsize=(40,10))
sns.pointplot(x=india.index,y=india.Confirmed,color='green')
sns.pointplot(x=china.index,y=china.Confirmed,color='blue')
sns.pointplot(x=us.index,y=us.Confirmed,color='yellow')
ax.set(xlabel='No of days',ylabel='Confirmed')

plt.xticks(rotation=90)
plt.show()

In [ ]:
#Model building using Facebook Prophet Library
from prophet import Prophet

In [ ]:
data=pd.read_csv('covid.csv',parse_dates=['Date'])

In [ ]:
data['Active'].sum()

In [ ]:
confirmed_d=data.groupby('Date')['Confirmed'].sum().reset_index()

In [ ]:
confirmed_d.columns=['ds','y']
confirmed_d['ds']=pd.to_datetime(confirmed_d['ds'])
confirmed_d.tail()

In [ ]:
deaths_d=data.groupby('Date')['Deaths'].sum().reset_index()
deaths_d.columns=['ds','y']
deaths_d['ds']=pd.to_datetime(deaths_d['ds'])
deaths_d.tail()


In [ ]:
m=Prophet()
m.fit(deaths_d)

In [ ]:
future=m.make_future_dataframe(periods=7)
future.tail()

In [ ]:
forecast=m.predict(future)
forecast[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
death_plot=m.plot(forecast)

In [ ]:
k=Prophet()
k.fit(confirmed_d)
future_confirmed=k.make_future_dataframe(periods=7)

In [ ]:

forecast_confirmed=k.predict(future_confirmed)
confirmed_plot=k.plot(forecast_confirmed)

In [ ]:
forecast_confirmed[['ds','yhat','yhat_lower','yhat_upper']].tail()

In [ ]:
confirmed_d.tail()